In [1]:
#importing all usefull lib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, train_test_split, KFold, cross_val_predict
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
%matplotlib inline
from sklearn.linear_model import LinearRegression, RidgeCV, Lasso, ElasticNetCV, BayesianRidge, LassoLarsIC
from sklearn.metrics import mean_squared_error, make_scorer 
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
import math

In [2]:
from datetime import datetime
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
from scipy import stats

In [3]:
train_dataRaw = pd.read_csv("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/ReferenceCode/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/ReferenceCode/house-prices-advanced-regression-techniques/test.csv")

In [4]:
Sale_Price=train_dataRaw.iloc[:,80]
Sale_Price.shape

(1460,)

In [5]:
train=train_dataRaw.drop(["SalePrice"],axis=1)
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [6]:
data= pd.concat([train,test], keys=['x', 'y'])#here X is training data and Y testing data
data=data.drop(["Id"],axis=1)
data.shape

(2919, 79)

In [7]:
tmp=data.loc["x"]

In [8]:
data=data.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 
                                       50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 
                                       80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 
                                       150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},
                       "MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
                                   7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
                      })

# missing_data

缺失值处理

In [9]:
total_null = data.isnull().sum().sort_values(ascending=False) #First sum and order all null values for each variable
percentage = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False) #Get the percentage
missing_data = pd.concat([total_null, percentage], axis=1, keys=['Total', 'Percentage'])

In [10]:
data = data.drop((missing_data[missing_data["Percentage"] > 0.05]).index,1)

In [11]:
num_col=data._get_numeric_data().columns.tolist()
cat_col=set(data.columns)-set(num_col)

In [12]:
for col in num_col:
    data[col].fillna(data[col].mean(),inplace=True)

In [13]:
for col in cat_col:
    data[col].fillna(data[col].mode()[0],inplace=True)

# Feature Engineering Begins

## 删除highly skewed Data

In [14]:
#delete highly skew category data in numerical variables
train_cat = data[cat_col]
pct=[]
for ix in train_cat.columns:
    temp=train_cat[ix].describe()
    print(temp.describe)
    break
    pct.append(temp["freq"]/temp["count"])
skewData=pd.DataFrame(pct,index=train_cat.columns,columns=["skewness"])
skewData=skewData.sort_values(by="skewness",ascending=False)
print (skewData)
data = data.drop((skewData[skewData['skewness'] >= 0.95]).index,1) 
data.shape

<bound method NDFrame.describe of count        2919
unique         15
top       VinylSd
freq         1026
Name: Exterior1st, dtype: object>
              skewness
Exterior1st        NaN
RoofMatl           NaN
Functional         NaN
SaleCondition      NaN
MoSold             NaN
MSSubClass         NaN
Utilities          NaN
BsmtCond           NaN
BldgType           NaN
LandSlope          NaN
Condition2         NaN
Electrical         NaN
CentralAir         NaN
Neighborhood       NaN
SaleType           NaN
Condition1         NaN
ExterQual          NaN
LandContour        NaN
PavedDrive         NaN
Exterior2nd        NaN
MasVnrType         NaN
LotConfig          NaN
KitchenQual        NaN
BsmtQual           NaN
ExterCond          NaN
Street             NaN
MSZoning           NaN
BsmtExposure       NaN
BsmtFinType2       NaN
LotShape           NaN
RoofStyle          NaN
Heating            NaN
HeatingQC          NaN
BsmtFinType1       NaN
HouseStyle         NaN
Foundation         NaN


(2919, 68)

In [15]:
train_data=data

In [16]:
#2.3: Simplifications of existing categorical 
train_data["Condition1"] = train_data.Condition1.replace({"RRNe" : "Other", 
                                                  "RRNn" : "Other","PosA" : "Other", 
                                                   "RRAe" : "Other"
                                                  })
train_data["Electrical"] = train_data.Electrical.replace({"Mix" : "Other", 
                                                  "FuseP" : "Other"
                                                  })
train_data["Exterior1st"] = train_data.Exterior1st.replace({"AsphShn" : "Other", 
                                                  "CBlock" : "Other","ImStucc" : "Other", 
                                                   "BrkComm" : "Other","Stone" : "Other"
                                                  })
train_data["Exterior2nd"] = train_data.Exterior2nd.replace({"CBlock" : "Other", 
                                                  "AsphShn" : "Other","Stone" : "Other", 
                                                   "Brk Cmn" : "Other","ImStucc" : "Other"
                                                  })
train_data["Foundation"] = train_data.Foundation.replace({"Wood" : "Other", 
                                                  "Stone" : "Other"
                                                  })
# train_data["GarageType"] = train_data.GarageType.replace({"2Types" : "Other", 
#                                                   "CarPort" : "Other"
#                                                   })
train_data["HouseStyle"] = train_data.HouseStyle.replace({"2.5Fin" : "Other", 
                                                  "2.5Unf" : "Other",
                                                  "1.5Unf" : "Other"
                                                  })
train_data["LotConfig"] = train_data.LotConfig.replace({"FR3" : "FR2"
                                                  })
train_data["MSSubClass"] = train_data.MSSubClass.replace({"SC40" : "Other", 
                                                  "SC180" : "Other",
                                                  "SC45" : "Other",
                                                  "SC75" : "Other"
                                                  })
train_data["MSZoning"] = train_data.MSZoning.replace({"C (all)" : "Other", 
                                                  "RH" : "Other"
                                                  })
train_data["Neighborhood"] = train_data.Neighborhood.replace({"Blueste" : "Other", 
                                                  "NPkVill" : "Other",
                                                  "Veenker" : "Other"
                                                  })
train_data["RoofStyle"] = train_data.RoofStyle.replace({"Shed" : "Other", 
                                                  "Mansard" : "Other",
                                                  "Gambrel" : "Other",
                                                  "Flat" : "Other"
                                                  })
train_data["SaleCondition"] = train_data.SaleCondition.replace({"AdjLand" : "Other", 
                                                  "Alloca" : "Other"
                                                  })
train_data["SaleType"] = train_data.SaleType.replace({"Con" : "Other", 
                                                  "Oth" : "Other",
                                                  "CWD" : "Other", "ConLI" : "Other",
                                                  "ConLw" : "Other","ConLD" : "Other"
                                                  })




In [17]:
# train_data["OverallQual-s2"] = train_data["OverallQual"] ** 2
# train_data["AllSF-2"] = train_data["AllSF"] ** 2
# train_data["AllFlrsSF-2"] = train_data["AllFlrsSF"] ** 2
# train_data["GrLivArea-2"] = train_data["GrLivArea"] ** 2
# # train_data["SimplOverallQual-s2"] = train_data["SimplOverallQual"] ** 2
# train_data["ExterQual-2"] = train_data["ExterQual"] ** 2
# train_data["GarageCars-2"] = train_data["GarageCars"] ** 2
# train_data["TotalBath-2"] = train_data["TotalBath"] ** 2
# train_data["KitchenQual-2"] = train_data["KitchenQual"] ** 2

In [18]:
# #2.4 Combinations of existing features
# # Overall quality of the house
# train_data["OverallGrade"] = train_data["OverallQual"] * train_data["OverallCond"]
# # Overall quality of the exterior
# train_data["ExterGrade"] = train_data["ExterQual"] * train_data["ExterCond"]
# # Overall kitchen score
# train_data["KitchenScore"] = train_data["KitchenAbvGr"] * train_data["KitchenQual"]
# # Total number of bathrooms
# train_data["TotalBath"] = train_data["BsmtFullBath"] + (0.5 * train_data["BsmtHalfBath"]) + \
# train_data["FullBath"] + (0.5 * train_data["HalfBath"])
# train_data.drop(["BsmtFullBath","BsmtHalfBath","FullBath","HalfBath"],axis=1)
# # Total SF for house (incl. basement)
# train_data["AllSF"] = train_data["GrLivArea"] + train_data["TotalBsmtSF"]
# train_data.drop(["GrLivArea","TotalBsmtSF"],axis=1)
# # Total SF for 1st + 2nd floors
# train_data["AllFlrsSF"] = train_data["1stFlrSF"] + train_data["2ndFlrSF"]
# train_data.drop(["1stFlrSF","2ndFlrSF"],axis=1)
# # Total SF for porch
# train_data["AllPorchSF"] = train_data["OpenPorchSF"] + train_data["EnclosedPorch"] + \
# train_data["3SsnPorch"] + train_data["ScreenPorch"]
# train_data.drop(["OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch"],axis=1)
# # House completed before sale or not
# train_data["BoughtOffPlan"] = train_data.SaleCondition.replace({"Abnorml" : 0, "Alloca" : 0, "AdjLand" : 0,"Family" : 0, "Normal" : 0, "Partial" : 1})

In [19]:
# Differentiate numerical features and categorical features
categorical_features = train_data.select_dtypes(include = ["object"]).columns
numerical_features = train_data.select_dtypes(exclude = ["object"]).columns
# numerical_features = numerical_features.drop("SalePrice")
print("Numerical features : " + str(len(numerical_features)))
print("Categorical features : " + str(len(categorical_features)))
train_num = train_data[numerical_features]
train_cat = train_data[categorical_features]
print("NAs for numerical features in train : " + str(train_num.isnull().values.sum()))
train_num = train_num.fillna(train_num.median())
print("Remaining NAs for numerical features in train : " + str(train_num.isnull().values.sum()))

#2.6 turn category features to dummy
print("NAs for categorical features in train : " + str(train_cat.isnull().values.sum()))
train_cat = pd.get_dummies(train_cat, drop_first=True)
print("Remaining NAs for categorical features in train : " + str(train_cat.isnull().values.sum()))

#2.7 join categorical and numerical features 
train_Data_New = pd.concat([train_num, train_cat], axis = 1)
print("New number of features : " + str(train_Data_New.shape[1]))

#2.8 remove collinear columns
# Create correlation matrix
corr_matrix = train_Data_New.corr().abs()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
# Drop features 
train_Data_New.drop(to_drop, axis=1, inplace=True)
data=train_Data_New

Numerical features : 32
Categorical features : 36
NAs for numerical features in train : 0
Remaining NAs for numerical features in train : 0
NAs for categorical features in train : 0
Remaining NAs for categorical features in train : 0
New number of features : 207


In [20]:
# bjadsfk=

## droping some unnecessary cat_features bcoz they have 80% + same value and 20% - defertnt values so they can't effect score【存疑】

In [21]:
# data=train_data

In [22]:

df=data.drop([
               "GarageArea","BsmtUnfSF","3SsnPorch","MiscVal"],axis=1)
# new   


In [23]:
corrmat = train_dataRaw.corr()
top_corr_features = corrmat.index[abs(corrmat["SalePrice"])>0.5]

In [24]:
df=df.drop(["YrSold"],axis=1)

In [25]:
df.shape

(2919, 195)

# END OF FEATURE ENGINEERING

In [26]:
X=pd.get_dummies(df)
X.shape

(2919, 195)

In [27]:
# djstra=dsakjf

In [28]:
scalerX = MinMaxScaler(feature_range=(0, 1))
X[X.columns] = scalerX.fit_transform(X[X.columns])

In [29]:
#Training data after preproscing

Train_data=X.loc["x"]
Train_data.shape

(1460, 195)

In [30]:
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model, X=Train_data):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kfolds))
    return (rmse)

In [31]:
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

In [32]:
ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=alphas_alt, cv=kfolds))
lasso = make_pipeline(RobustScaler(), LassoCV(max_iter=1e7, alphas=alphas2, random_state=42, cv=kfolds))
elasticnet = make_pipeline(RobustScaler(), ElasticNetCV(max_iter=1e7, alphas=e_alphas, cv=kfolds, l1_ratio=e_l1ratio))                                
svr = make_pipeline(RobustScaler(), SVR(C= 20, epsilon= 0.008, gamma=0.0003,))

In [33]:
# score = cv_rmse(ridge)
# score = cv_rmse(lasso)
# print("LASSO: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

# score = cv_rmse(elasticnet)
# print("elastic net: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

# score = cv_rmse(svr)
# print("SVR: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

In [34]:
#Testing data after preproscing
Test_data=X.loc["y"]
Test_data.shape

(1459, 195)

In [35]:
#here we add salePrice column in traning data
Train_data.insert(2,column="SalePrice",value=Sale_Price)
Train_data.head()

,LotArea,OverallQual,SalePrice,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,TotalBsmtSF,...,MoSold_May,MoSold_Nov,MoSold_Oct,MoSold_Sep,SaleType_New,SaleType_Other,SaleType_WD,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Other
0,0.033420,0.666667,208500,0.500,0.949275,0.883333,0.12250,0.125089,0.0,0.140098,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.038795,0.555556,181500,0.875,0.753623,0.433333,0.00000,0.173281,0.0,0.206547,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.046507,0.666667,223500,0.500,0.934783,0.866667,0.10125,0.086109,0.0,0.150573,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.038561,0.666667,140000,0.500,0.311594,0.333333,0.00000,0.038271,0.0,0.123732,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.060576,0.777778,250000,0.500,0.927536,0.833333,0.21875,0.116052,0.0,0.187398,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [36]:
x=Train_data.drop(["SalePrice"],axis=True)
y=Train_data["SalePrice"]

In [37]:
# print('Lasso')
# lasso_model_full_data = lasso.fit(x, y)
# print('Ridge')
# ridge_model_full_data = ridge.fit(x, y)
# print('Svr')
# svr_model_full_data = svr.fit(x, y)

## train_test_split

In [38]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.30,random_state=40)

In [39]:
from sklearn.ensemble import RandomForestRegressor
rfr=RandomForestRegressor(n_estimators = 50,random_state=40,
                          min_impurity_decrease=0.002,min_weight_fraction_leaf=0.001,min_samples_split=5)
rfr.fit(x_train,y_train)
y_predictrfc = rfr.predict(x_test)

#here we can check our model score
print(rfr.score(x_test,y_test))

0.8788879766107057


In [40]:
rmse = math.sqrt(mean_squared_error(y_test, rfr.predict(x_test)))

print("mear squares error :",rmse)

mear squares error : 25111.505338925122


In [41]:
from sklearn.tree import DecisionTreeRegressor
dtr=DecisionTreeRegressor(random_state=140,min_samples_split=5,min_impurity_decrease=0.002,min_weight_fraction_leaf=0.001)
dtr.fit(x_train,y_train)

#u can also use GridSearchCV / random Searchcv for hyperperameter tuning
print(dtr.score(x_test,y_test))


0.7402935013636911


In [42]:
rmse = math.sqrt(mean_squared_error(y_test, dtr.predict(x_test)))
print("RMSE:",rmse)

RMSE: 36772.27729596568


## GBoost parameters

In [43]:
# GBoost = GradientBoostingRegressor(n_estimators=5000, learning_rate=0.05,
#                                    max_depth=4, max_features='sqrt',
#                                    min_samples_leaf=15, min_samples_split=10, 
#                                    loss='huber', random_state =5)
# #RMSE estimated through the partition of the train set
# GBoost.fit(x_train, y_train)

# print("RMSE: %.4f" % rmse)

In [44]:
GBoost= GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =42)
                                   #RMSE estimated through the partition of the train set
GBoost.fit(x_train, y_train)

print("RMSE: %.4f" % rmse)
                                   

RMSE: 36772.2773


In [45]:
print(GBoost.score(x_test,y_test))

0.9204853720103662


In [46]:
lightgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=5000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       #min_data_in_leaf=2,
                                       #min_sum_hessian_in_leaf=11
                                       )
lightgbm.fit(x_train,y_train)
print(lightgbm.score(x_test,y_test))

[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75
0.9062602968991099


In [47]:
xgboost = XGBRegressor(learning_rate=0.01, n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                 reg_alpha=0.00006)
xgboost.fit(x_train,y_train) 
print(xgboost.score(x_test,y_test))                            

[18:52:36] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
0.8925783006046037
/usr/local/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [48]:
# stack
# stack_gen = StackingCVRegressor(regressors=( xgboost, GBoost, lightgbm),meta_regressor=GBoost,use_features_in_secondary=True)

In [49]:
# stack_gen.fit(x_train,y_train)
# print(stack_gen.score(x_test,y_test))   

In [50]:
#here we see test data here one column is missing that is Saleprice bcoz that is need to predict
Test_data.head()

,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,TotalBsmtSF,1stFlrSF,...,MoSold_May,MoSold_Nov,MoSold_Oct,MoSold_Sep,SaleType_New,SaleType_Other,SaleType_WD,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Other
0,0.048246,0.444444,0.625,0.644928,0.183333,0.0000,0.082920,0.094364,0.144354,0.118042,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.060609,0.555556,0.625,0.623188,0.133333,0.0675,0.163536,0.000000,0.217512,0.208990,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.058566,0.444444,0.500,0.905797,0.800000,0.0000,0.140149,0.000000,0.151882,0.124764,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.040562,0.555556,0.625,0.913043,0.800000,0.0125,0.106662,0.000000,0.151555,0.124344,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.017318,0.777778,0.500,0.869565,0.700000,0.0000,0.046598,0.000000,0.209493,0.198698,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [51]:
Test_data.shape

(1459, 195)

In [52]:
# y_model_prerfc = GBoost.predict(Test_data)
# y_model_prerfc=dtr.predict(Test_data)
# y_model_prerfc = stack_gen.predict(Test_data)
y_model_prerfc=xgboost.predict(Test_data)

/usr/local/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [53]:
y_model_prerfc=np.around(y_model_prerfc,0)

In [54]:
prediction=np.array(y_model_prerfc).tolist()
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [55]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [56]:
# test=test.drop(["SalePrice"],axis=1)

In [57]:
test.insert(1,column="SalePrice",value=prediction)
test.head()

,Id,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,117377.0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,164237.0,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,181915.0,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,193190.0,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,185866.0,120,RL,43.0,5005,Pave,NaN,IR1,HLS,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [58]:
predict_sub=test.drop(test.iloc[:,2:],axis=1)
predict_sub.head()

,Id,SalePrice
0,1461,117377.0
1,1462,164237.0
2,1463,181915.0
3,1464,193190.0
4,1465,185866.0


In [59]:
predict_sub.shape


(1459, 2)

In [60]:
predict_sub.to_csv('Home_predictionsTest.csv',index=False)